In [2]:
import mne

# 设置编码为 'latin1'
raw = mne.io.read_raw_edf('糖尿病认知障碍与对照脑电数据/认知障碍/陈艳杰.edf', preload=True, encoding='latin1')
import warnings

# 忽略 RuntimeWarning 警告
warnings.filterwarnings("ignore", category=RuntimeWarning)

# 查看滤波信息
print("Highpass filter:", raw.info['highpass'])
print("Lowpass filter:", raw.info['lowpass'])

# 查看注释信息
print("Annotations:", raw.annotations)

# 打印完整的基本信息
print(raw.info)



Extracting EDF parameters from /home/lyq/Desktop/DL/NEW_STCGRU/糖尿病认知障碍与对照脑电数据/认知障碍/陈艳杰.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 236999  =      0.000 ...   473.998 secs...
Highpass filter: 0.0
Lowpass filter: 250.0
Annotations: <Annotations | 4 segments: +0.000000 (1), +1.560000 (1), A1+A2 OFF (1), ...>
<Info | 8 non-empty values
 bads: []
 ch_names: EEG Fp1-Ref, EEG Fp2-Ref, EEG F3-Ref, EEG F4-Ref, EEG C3-Ref, ...
 chs: 43 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2024-01-11 10:31:24 UTC
 nchan: 43
 projs: []
 sfreq: 500.0 Hz
 subject_info: 3 items (dict)
>


In [ ]:
data = raw.get_data()

In [1]:
import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
import torch
import model
stcgru = model.STCGRU().to(device)
test = torch.randn( 1, 19, 2500).to(device)
output = stcgru(test)

out1 shape: torch.Size([1, 1, 19, 61])
out2 shape: torch.Size([1, 1, 19, 67])


In [ ]:
print(stcgru)